# IOI Circuit Replication Validation

**End-to-End Validation Against "Interpretability in the Wild" (Wang et al. 2022)**

This notebook validates our implementation of the Indirect Object Identification (IOI) circuit discovery against the original paper's findings.

## Paper Reference
Wang, K., et al. (2022). Interpretability in the Wild: a Circuit for Indirect Object Identification in GPT-2 small. *ICLR 2023*.

## Implementation Reference
ARENA 1.4.1: Indirect Object Identification

---

## Setup

In [ ]:
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime
import torch

# Add project root to path
sys.path.append('..')

# Import our modules
from src.data.dataset import generate_ioi_dataset, load_ioi_dataset
from src.model.model_loader import load_ioi_model
from src.analysis.ioi_baseline import run_baseline, compute_logit_diff
from src.analysis.activation_patching import (
    patch_all_layers,
    patch_all_heads,
    analyze_example_patching
)
from src.analysis.attention_analysis import (
    find_all_ioi_heads,
    analyze_duplicate_token_attention,
    analyze_s_inhibition_attention,
    analyze_name_mover_attention
)
from src.analysis.path_patching import analyze_ioi_circuit_paths
from src.analysis.circuit_discovery import (
    discover_ioi_circuit,
    validate_circuit,
    print_circuit_summary
)
from src.analysis.logit_attribution import (
    compare_io_vs_s_attribution,
    plot_logit_attribution,
    analyze_circuit_with_dla
)

# Plotting configuration
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✓ Imports successful")
print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

## Expected Results from Original Paper

### Key Findings to Validate:

1. **Baseline Performance**
   - Model accuracy on IOI task: ~95% (paper reports high accuracy)
   - Logit difference (IO - S): Positive and substantial (typically 5-10)

2. **Circuit Components** (from paper Fig 2 & 3)
   - **Duplicate Token Heads** (early layers 0-3): L0H1, L2H2, L3H0, etc.
   - **S-Inhibition Heads** (middle layers 7-8): L7H3, L7H9, L8H6, L8H10
   - **Name Mover Heads** (late layers 9-11): L9H6, L9H9, L10H0, L10H2, L11H10

3. **Activation Patching Effects**
   - Name mover heads: High positive effect (>0.5)
   - S-inhibition heads: Moderate positive effect (0.2-0.5)
   - Duplicate token heads: Low-moderate effect (0.1-0.3)

4. **Path Patching**
   - Strong paths: Duplicate → S-inhibition
   - Strong paths: S-inhibition → Name mover
   - Some direct paths: Duplicate → Name mover

5. **Direct Logit Attribution**
   - Circuit heads account for 80-95% of logit difference
   - Name movers: Large positive contribution to IO token
   - S-inhibition: Large negative contribution to S token

---
## Phase 1: Data Generation and Model Loading

In [ ]:
print("="*80)
print("PHASE 1: DATA GENERATION AND MODEL LOADING")
print("="*80)

# Generate datasets
print("\n1. Generating ABBA dataset (clean prompts)...")
generate_ioi_dataset(n_examples=500, template="ABBA", seed=42)
abba_dataset = load_ioi_dataset("../data/ioi_abba.json")
print(f"   Generated {len(abba_dataset)} ABBA examples")

print("\n2. Generating ABC dataset (corrupted prompts)...")
generate_ioi_dataset(n_examples=500, template="ABC", seed=42)
abc_dataset = load_ioi_dataset("../data/ioi_abc.json")
print(f"   Generated {len(abc_dataset)} ABC examples")

# Load model
print("\n3. Loading GPT-2 Small with TransformerLens...")
device = "cuda" if torch.cuda.is_available() else "cpu"
result = load_ioi_model(device=device)
model = result["model"]
config = result["config"]

print(f"   Model: {config['model_name']}")
print(f"   Layers: {config['n_layers']}")
print(f"   Heads: {config['n_heads']}")
print(f"   Device: {config['device']}")

print("\n✓ Phase 1 complete")

---
## Phase 2: Baseline Performance

In [ ]:
print("="*80)
print("PHASE 2: BASELINE PERFORMANCE")
print("="*80)

# Run baseline on sample
print("\nRunning baseline analysis on 100 examples...")
baseline_results = run_baseline(
    model,
    "../data/ioi_abba.json",
    max_examples=100
)

# Extract metrics
accuracy = baseline_results["accuracy"]
mean_logit_diff = baseline_results["mean_logit_diff"]
std_logit_diff = baseline_results["std_logit_diff"]

print("\n" + "="*60)
print("BASELINE RESULTS")
print("="*60)
print(f"Accuracy: {accuracy:.1%}")
print(f"Mean Logit Diff (IO - S): {mean_logit_diff:.3f} ± {std_logit_diff:.3f}")
print("="*60)

# Validation against paper
validation_report = []

print("\n📊 VALIDATION CHECK 1: Baseline Performance")
if accuracy >= 0.90:
    print(f"   ✓ PASS: Accuracy {accuracy:.1%} >= 90% (paper expects ~95%)")
    validation_report.append({"test": "Baseline Accuracy", "status": "PASS", "value": f"{accuracy:.1%}"})
else:
    print(f"   ✗ FAIL: Accuracy {accuracy:.1%} < 90%")
    validation_report.append({"test": "Baseline Accuracy", "status": "FAIL", "value": f"{accuracy:.1%}"})

if mean_logit_diff >= 3.0:
    print(f"   ✓ PASS: Mean logit diff {mean_logit_diff:.3f} >= 3.0")
    validation_report.append({"test": "Logit Difference", "status": "PASS", "value": f"{mean_logit_diff:.3f}"})
else:
    print(f"   ✗ FAIL: Mean logit diff {mean_logit_diff:.3f} < 3.0")
    validation_report.append({"test": "Logit Difference", "status": "FAIL", "value": f"{mean_logit_diff:.3f}"})

print("\n✓ Phase 2 complete")

---
## Phase 3: Activation Patching

In [ ]:
print("="*80)
print("PHASE 3: ACTIVATION PATCHING")
print("="*80)

# Use first example for detailed analysis
example = abba_dataset[0]
clean_prompt = example["prompt"]
print(f"\nClean prompt: {clean_prompt}")

# Create corrupted version
corrupted_prompt = clean_prompt.replace(
    f", {example['io_name']} gave",
    f", {example['s_name']} gave"
)
print(f"Corrupted prompt: {corrupted_prompt}")

clean_tokens = model.to_tokens(clean_prompt)
corrupted_tokens = model.to_tokens(corrupted_prompt)

io_token_id = model.to_single_token(" " + example["io_name"])
s_token_id = model.to_single_token(" " + example["s_name"])

# Patch all layers
print("\n1. Patching residual stream at all layers...")
layer_effects = patch_all_layers(
    model, clean_tokens, corrupted_tokens,
    io_token_id, s_token_id
)

# Plot layer effects
plt.figure(figsize=(12, 5))
plt.plot(range(len(layer_effects)), layer_effects, marker='o')
plt.xlabel('Layer')
plt.ylabel('Patching Effect')
plt.title('Activation Patching by Layer\n(Effect of restoring clean activations)')
plt.grid(True, alpha=0.3)
plt.axhline(y=0.5, color='r', linestyle='--', alpha=0.5, label='High Effect (>0.5)')
plt.legend()
plt.tight_layout()
plt.savefig('../results/activation_patching_layers.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nTop 3 most important layers:")
top_layers = sorted(enumerate(layer_effects), key=lambda x: x[1], reverse=True)[:3]
for layer, effect in top_layers:
    print(f"   Layer {layer}: {effect:.3f}")

# Patch all heads
print("\n2. Patching all attention heads...")
head_effects = patch_all_heads(
    model, clean_tokens, corrupted_tokens,
    io_token_id, s_token_id
)

# Reshape for heatmap
n_layers = model.cfg.n_layers
n_heads = model.cfg.n_heads
head_matrix = np.zeros((n_layers, n_heads))
for (layer, head), effect in head_effects.items():
    head_matrix[layer, head] = effect

# Plot heatmap
plt.figure(figsize=(14, 8))
sns.heatmap(
    head_matrix,
    cmap='RdYlGn',
    center=0,
    vmin=-0.2,
    vmax=1.0,
    cbar_kws={'label': 'Patching Effect'},
    xticklabels=range(n_heads),
    yticklabels=range(n_layers)
)
plt.xlabel('Head')
plt.ylabel('Layer')
plt.title('Activation Patching by Head\n(Higher = More Important for IOI Task)')
plt.tight_layout()
plt.savefig('../results/activation_patching_heads.png', dpi=150, bbox_inches='tight')
plt.show()

# Find top heads
print(f"\nTop 10 most important heads:")
top_heads = sorted(head_effects.items(), key=lambda x: x[1], reverse=True)[:10]
for (layer, head), effect in top_heads:
    print(f"   L{layer}H{head}: {effect:.3f}")

print("\n✓ Phase 3 complete")

---
## Phase 4: Attention Pattern Analysis

In [ ]:
print("="*80)
print("PHASE 4: ATTENTION PATTERN ANALYSIS")
print("="*80)

# Find all IOI heads using attention patterns
print("\nFinding IOI circuit heads using attention patterns...")
print("(Using 50 examples for robustness)\n")

ioi_heads = find_all_ioi_heads(
    model,
    "../data/ioi_abba.json",
    max_examples=50,
    dup_threshold=0.4,
    s_inhibition_threshold=0.3,
    name_mover_threshold=0.3
)

duplicate_heads = ioi_heads["duplicate_token_heads"]
s_inhibition_heads = ioi_heads["s_inhibition_heads"]
name_mover_heads = ioi_heads["name_mover_heads"]

print("\n" + "="*60)
print("DISCOVERED CIRCUIT HEADS")
print("="*60)
print(f"\nDuplicate Token Heads ({len(duplicate_heads)}):")
for layer, head in sorted(duplicate_heads):
    print(f"   L{layer}H{head}")

print(f"\nS-Inhibition Heads ({len(s_inhibition_heads)}):")
for layer, head in sorted(s_inhibition_heads):
    print(f"   L{layer}H{head}")

print(f"\nName Mover Heads ({len(name_mover_heads)}):")
for layer, head in sorted(name_mover_heads):
    print(f"   L{layer}H{head}")

print("\n" + "="*60)

# Compare with paper's reported heads
paper_name_movers = [(9, 6), (9, 9), (10, 0), (10, 2)]
paper_s_inhibition = [(7, 3), (7, 9), (8, 6), (8, 10)]

print("\n📊 VALIDATION CHECK 2: Circuit Head Discovery")

# Check name movers
nm_overlap = len(set(name_mover_heads) & set(paper_name_movers))
print(f"\nName Mover Heads:")
print(f"   Paper reports: {paper_name_movers}")
print(f"   We found: {sorted(name_mover_heads)}")
print(f"   Overlap: {nm_overlap}/{len(paper_name_movers)}")
if nm_overlap >= 2:
    print(f"   ✓ PASS: Found {nm_overlap} of paper's key name movers")
    validation_report.append({"test": "Name Mover Discovery", "status": "PASS", "value": f"{nm_overlap}/{len(paper_name_movers)}"})
else:
    print(f"   ✗ FAIL: Only found {nm_overlap} of paper's key name movers")
    validation_report.append({"test": "Name Mover Discovery", "status": "FAIL", "value": f"{nm_overlap}/{len(paper_name_movers)}"})

# Check S-inhibition
si_overlap = len(set(s_inhibition_heads) & set(paper_s_inhibition))
print(f"\nS-Inhibition Heads:")
print(f"   Paper reports: {paper_s_inhibition}")
print(f"   We found: {sorted(s_inhibition_heads)}")
print(f"   Overlap: {si_overlap}/{len(paper_s_inhibition)}")
if si_overlap >= 2:
    print(f"   ✓ PASS: Found {si_overlap} of paper's key S-inhibition heads")
    validation_report.append({"test": "S-Inhibition Discovery", "status": "PASS", "value": f"{si_overlap}/{len(paper_s_inhibition)}"})
else:
    print(f"   ✗ FAIL: Only found {si_overlap} of paper's key S-inhibition heads")
    validation_report.append({"test": "S-Inhibition Discovery", "status": "FAIL", "value": f"{si_overlap}/{len(paper_s_inhibition)}"})

# Check duplicate token heads are in early layers
dup_in_early = sum(1 for layer, head in duplicate_heads if layer <= 3)
print(f"\nDuplicate Token Heads:")
print(f"   We found: {sorted(duplicate_heads)}")
print(f"   In early layers (0-3): {dup_in_early}/{len(duplicate_heads)}")
if dup_in_early >= len(duplicate_heads) * 0.6:
    print(f"   ✓ PASS: Most duplicate token heads are in early layers")
    validation_report.append({"test": "Duplicate Head Layers", "status": "PASS", "value": f"{dup_in_early}/{len(duplicate_heads)}"})
else:
    print(f"   ✗ FAIL: Not enough duplicate token heads in early layers")
    validation_report.append({"test": "Duplicate Head Layers", "status": "FAIL", "value": f"{dup_in_early}/{len(duplicate_heads)}"})

print("\n✓ Phase 4 complete")

---
## Phase 5: Path Patching Analysis

In [ ]:
print("="*80)
print("PHASE 5: PATH PATCHING ANALYSIS")
print("="*80)

if len(duplicate_heads) > 0 and len(s_inhibition_heads) > 0 and len(name_mover_heads) > 0:
    print("\nAnalyzing paths between circuit components...")
    
    path_results = analyze_ioi_circuit_paths(
        model,
        clean_tokens,
        corrupted_tokens,
        duplicate_heads[:3],  # Use top 3 of each type
        s_inhibition_heads[:3],
        name_mover_heads[:3],
        io_token_id,
        s_token_id
    )
    
    # Analyze duplicate → S-inhibition paths
    print("\n1. Duplicate Token → S-Inhibition Paths:")
    dup_to_si = path_results["dup_to_s_inhibition"]["effect_matrix"]
    max_dup_si = np.max(dup_to_si)
    print(f"   Max effect: {max_dup_si:.3f}")
    print(f"   Mean effect: {np.mean(dup_to_si):.3f}")
    
    # Analyze S-inhibition → Name mover paths
    print("\n2. S-Inhibition → Name Mover Paths:")
    si_to_nm = path_results["s_inhibition_to_name_mover"]["effect_matrix"]
    max_si_nm = np.max(si_to_nm)
    print(f"   Max effect: {max_si_nm:.3f}")
    print(f"   Mean effect: {np.mean(si_to_nm):.3f}")
    
    # Analyze duplicate → Name mover paths
    print("\n3. Duplicate Token → Name Mover Paths:")
    dup_to_nm = path_results["dup_to_name_mover"]["effect_matrix"]
    max_dup_nm = np.max(dup_to_nm)
    print(f"   Max effect: {max_dup_nm:.3f}")
    print(f"   Mean effect: {np.mean(dup_to_nm):.3f}")
    
    print("\n📊 VALIDATION CHECK 3: Path Importance")
    if max_si_nm > 0.3:
        print(f"   ✓ PASS: Found strong S-Inhibition → Name Mover path ({max_si_nm:.3f})")
        validation_report.append({"test": "SI→NM Path", "status": "PASS", "value": f"{max_si_nm:.3f}"})
    else:
        print(f"   ✗ FAIL: No strong S-Inhibition → Name Mover path")
        validation_report.append({"test": "SI→NM Path", "status": "FAIL", "value": f"{max_si_nm:.3f}"})
else:
    print("\n⚠ Skipping path patching - insufficient heads discovered")
    validation_report.append({"test": "Path Patching", "status": "SKIP", "value": "Insufficient heads"})

print("\n✓ Phase 5 complete")

---
## Phase 6: Complete Circuit Discovery

In [ ]:
print("="*80)
print("PHASE 6: COMPLETE CIRCUIT DISCOVERY")
print("="*80)

print("\nRunning complete circuit discovery pipeline...")
print("(This combines attention patterns + activation patching + path patching)\n")

discovered_circuit = discover_ioi_circuit(
    model,
    "../data/ioi_abba.json",
    max_examples=30,
    head_threshold=0.35,
    path_threshold=0.25
)

# Print summary
print_circuit_summary(discovered_circuit)

# Save circuit
circuit_path = "../results/discovered_ioi_circuit.json"
with open(circuit_path, 'w') as f:
    # Convert tuples to lists for JSON serialization
    json_circuit = {
        "duplicate_token_heads": [list(h) for h in discovered_circuit["duplicate_token_heads"]],
        "s_inhibition_heads": [list(h) for h in discovered_circuit["s_inhibition_heads"]],
        "name_mover_heads": [list(h) for h in discovered_circuit["name_mover_heads"]],
        "critical_paths": [
            {**p, "from": list(p["from"]), "to": list(p["to"])}
            for p in discovered_circuit["critical_paths"]
        ],
        "head_effects": discovered_circuit["head_effects"],
        "metadata": discovered_circuit["metadata"]
    }
    json.dump(json_circuit, f, indent=2)
print(f"\n💾 Saved circuit to {circuit_path}")

# Validate circuit
print("\nValidating discovered circuit...")
validation_results = validate_circuit(
    model,
    "../data/ioi_abba.json",
    discovered_circuit,
    max_examples=50
)

print("\n✓ Phase 6 complete")

---
## Phase 7: Direct Logit Attribution

In [ ]:
print("="*80)
print("PHASE 7: DIRECT LOGIT ATTRIBUTION")
print("="*80)

print("\nComputing direct logit attribution for IO vs S tokens...")

# Use the same example from before
print(f"\nPrompt: {clean_prompt}")
print(f"IO token: {example['io_name']}")
print(f"S token: {example['s_name']}")

# Compare IO vs S attribution
comparison = compare_io_vs_s_attribution(
    model,
    clean_tokens,
    io_token_id,
    s_token_id
)

print(f"\nLogit difference (IO - S): {comparison['logit_diff']:.3f}")

# Show top contributors
print("\nTop 5 heads contributing to IO token:")
for layer, head, contrib in comparison["top_io_heads"][:5]:
    print(f"   L{layer}H{head}: {contrib:6.3f}")

print("\nTop 5 heads suppressing S token:")
for layer, head, contrib in comparison["top_s_suppression_heads"][:5]:
    print(f"   L{layer}H{head}: {contrib:6.3f}")

# Create visualization
plot_logit_attribution(
    comparison,
    save_path="../results/logit_attribution.png",
    top_n=15
)

# Analyze circuit with DLA
print("\nAnalyzing circuit with Direct Logit Attribution...")
dla_analysis = analyze_circuit_with_dla(
    model,
    clean_tokens,
    io_token_id,
    s_token_id,
    circuit_heads=discovered_circuit
)

circuit_percentage = dla_analysis["circuit_analysis"]["circuit_percentage"]

print("\n📊 VALIDATION CHECK 4: Circuit Attribution")
if circuit_percentage >= 0.60:
    print(f"   ✓ PASS: Circuit accounts for {circuit_percentage:.1%} of logit diff (≥60%)")
    validation_report.append({"test": "Circuit Attribution", "status": "PASS", "value": f"{circuit_percentage:.1%}"})
else:
    print(f"   ✗ FAIL: Circuit only accounts for {circuit_percentage:.1%} of logit diff")
    validation_report.append({"test": "Circuit Attribution", "status": "FAIL", "value": f"{circuit_percentage:.1%}"})

print("\n✓ Phase 7 complete")

---
## Final Validation Report

In [ ]:
print("="*80)
print("FINAL VALIDATION REPORT")
print("="*80)

# Create report dataframe
report_df = pd.DataFrame(validation_report)

# Display report
print("\n")
print(report_df.to_string(index=False))
print("\n")

# Count results
n_pass = sum(1 for r in validation_report if r["status"] == "PASS")
n_fail = sum(1 for r in validation_report if r["status"] == "FAIL")
n_skip = sum(1 for r in validation_report if r["status"] == "SKIP")
n_total = len(validation_report)

print("="*80)
print(f"RESULTS: {n_pass} PASS | {n_fail} FAIL | {n_skip} SKIP | {n_total} TOTAL")
print("="*80)

# Overall assessment
pass_rate = n_pass / max(n_total - n_skip, 1)

print("\n📊 OVERALL ASSESSMENT:\n")
if pass_rate >= 0.8:
    print("✅ EXCELLENT: Our implementation successfully replicates the IOI circuit!")
    print("   The discovered circuit matches the paper's key findings.")
elif pass_rate >= 0.6:
    print("✓ GOOD: Our implementation largely replicates the IOI circuit.")
    print("   Some minor discrepancies exist but overall structure is correct.")
elif pass_rate >= 0.4:
    print("⚠ PARTIAL: Our implementation partially replicates the IOI circuit.")
    print("   Significant discrepancies exist. Further investigation needed.")
else:
    print("❌ POOR: Our implementation does not match the paper's findings.")
    print("   Major issues detected. Review implementation carefully.")

# Save report
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
report_path = f"../results/validation_report_{timestamp}.csv"
report_df.to_csv(report_path, index=False)
print(f"\n💾 Saved detailed report to {report_path}")

# Create summary document
summary_path = f"../results/validation_summary_{timestamp}.txt"
with open(summary_path, 'w') as f:
    f.write("="*80 + "\n")
    f.write("IOI CIRCUIT REPLICATION VALIDATION SUMMARY\n")
    f.write("="*80 + "\n\n")
    f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Model: GPT-2 Small\n")
    f.write(f"Device: {device}\n\n")
    
    f.write("BASELINE PERFORMANCE\n")
    f.write("-" * 40 + "\n")
    f.write(f"Accuracy: {accuracy:.1%}\n")
    f.write(f"Mean Logit Diff: {mean_logit_diff:.3f} ± {std_logit_diff:.3f}\n\n")
    
    f.write("DISCOVERED CIRCUIT\n")
    f.write("-" * 40 + "\n")
    f.write(f"Duplicate Token Heads: {len(discovered_circuit['duplicate_token_heads'])}\n")
    f.write(f"S-Inhibition Heads: {len(discovered_circuit['s_inhibition_heads'])}\n")
    f.write(f"Name Mover Heads: {len(discovered_circuit['name_mover_heads'])}\n")
    f.write(f"Critical Paths: {len(discovered_circuit['critical_paths'])}\n\n")
    
    f.write("VALIDATION RESULTS\n")
    f.write("-" * 40 + "\n")
    f.write(report_df.to_string(index=False))
    f.write("\n\n")
    
    f.write(f"SUMMARY: {n_pass} PASS | {n_fail} FAIL | {n_skip} SKIP\n")
    f.write(f"Pass Rate: {pass_rate:.1%}\n")

print(f"💾 Saved summary to {summary_path}")

print("\n" + "="*80)
print("✓ VALIDATION COMPLETE")
print("="*80)

---
## Key Findings Summary

In [ ]:
print("="*80)
print("KEY FINDINGS SUMMARY")
print("="*80)

print("\n1. BASELINE PERFORMANCE")
print(f"   - Model accuracy: {accuracy:.1%}")
print(f"   - Mean logit difference: {mean_logit_diff:.3f}")
print(f"   - Paper expectation: ~95% accuracy, positive logit diff")

print("\n2. CIRCUIT COMPONENTS")
print(f"   - Duplicate Token Heads: {sorted(discovered_circuit['duplicate_token_heads'])}")
print(f"   - S-Inhibition Heads: {sorted(discovered_circuit['s_inhibition_heads'])}")
print(f"   - Name Mover Heads: {sorted(discovered_circuit['name_mover_heads'])}")

print("\n3. CIRCUIT VALIDATION")
if "circuit_analysis" in dla_analysis:
    print(f"   - Circuit logit diff contribution: {dla_analysis['circuit_analysis']['circuit_logit_diff']:.3f}")
    print(f"   - Total logit diff: {dla_analysis['circuit_analysis']['total_logit_diff']:.3f}")
    print(f"   - Circuit percentage: {dla_analysis['circuit_analysis']['circuit_percentage']:.1%}")
    print(f"   - Paper expectation: 80-95% from circuit heads")

print("\n4. COMPARISON WITH PAPER")
print(f"   - Name mover overlap: {nm_overlap}/{len(paper_name_movers)} key heads")
print(f"   - S-inhibition overlap: {si_overlap}/{len(paper_s_inhibition)} key heads")
print(f"   - Overall validation: {n_pass}/{n_total - n_skip} checks passed")

print("\n" + "="*80)

---
## Notes and Observations

### Expected Behavior:
1. The model should achieve >90% accuracy on IOI task
2. Circuit heads should be concentrated in expected layers:
   - Duplicate token heads: Early layers (0-3)
   - S-inhibition heads: Middle layers (7-8)
   - Name mover heads: Late layers (9-11)
3. Circuit heads should account for majority (>60%) of logit difference

### Potential Discrepancies:
- Exact heads may vary slightly depending on dataset and thresholds
- Some heads may have borderline effects and be included/excluded based on threshold
- Path effects can be noisy on single examples

### Next Steps:
1. If validation fails: Try different thresholds or more examples
2. Analyze specific discrepancies in detail
3. Consider running on additional dataset templates
4. Compare attention patterns qualitatively

---